In [2]:
import cogent3
import pandas as pd
import numpy as np
import paths

In [3]:
folder_in = paths.DATA_APES114
folder_out = paths.DATA_APES114

table = cogent3.load_table(folder_in + "homo_sapiens-114-gene_metadata.tsv")

table

homo_sapiens,3,,lncRNA,"lncRNA,retained_intron",11,185959733,186012561,1,NMRAL2P,"NmrA like redox sensor 2, pseudogene [Source:NCBI gene (formerly Entrezgene);Acc:344887]"
homo_sapiens,4,,lncRNA,lncRNA,83,3941043,3956353,-1,FAM86EP,"family with sequence similarity 86, member A pseudogene [Source:NCBI gene (formerly Entrezgene);Acc:348926]"
homo_sapiens,19,,lncRNA,"lncRNA,retained_intron",19,60950,72334,-1,WASH5P,"WASP family homolog 5, pseudogene [Source:NCBI gene (formerly Entrezgene);Acc:375690]"
homo_sapiens,17,,lncRNA,lncRNA,1,57045477,57046797,1,RNF126P1,ring finger protein 126 pseudogene 1 [Source:NCBI gene (formerly Entrezgene);Acc:376412]
homo_sapiens,16,,lncRNA,lncRNA,2,32877468,32885142,-1,SLC6A10P,"solute carrier family 6 member 10, pseudogene [Source:NCBI gene (formerly Entrezgene);Acc:386757]"
...,...,...,...,...,...,...,...,...,...,...
homo_sapiens,KI270744.1,,lncRNA,lncRNA,1,142830,150130,1,,novel transcript
homo_sapiens,KI270744.1,,lncRNA,lncRNA,1,65347,66807,1,,novel transcript
homo_sapiens,KI270744.1,,lncRNA,lncRNA,1,423,6215,-1,,novel transcript
homo_sapiens,KI270744.1,,lncRNA,lncRNA,18,5312,80720,1,,novel transcript
homo_sapiens,KI270744.1,,lncRNA,lncRNA,5,90022,112250,-1,,novel transcript


In [15]:
ref_species = "homo_sapiens"
chrom_coord = 22
strand = 1
coordanates_table = table.filtered(lambda x: x == chrom_coord, columns="seqid")

coordanates_table = coordanates_table[:, ["species", "seqid", "start", "stop", "strand"]]
coordanates_table = coordanates_table.sorted(columns=["start", "stop"])
#write a sample table for genome 22
coordanates_table.write(folder_out + "chrom" + str(chrom_coord) + "-intragenic.tsv")


In [16]:
start_stop_overlapping = coordanates_table.to_list(columns=["start", "stop"])
start_stop_nonoverlapping = cogent3.util.misc.get_merged_overlapping_coords(start_stop_overlapping)
#using subsample to speed analysis, remove next line when doing the final analysis
start_stop_nonoverlapping = start_stop_nonoverlapping

inter_genic = []
last_end = 0
for start, stop in start_stop_nonoverlapping:
   inter_genic.append((ref_species, chrom_coord, last_end, start-1, strand))
   last_end = stop

intergen_tab = cogent3.make_table(header=["species", "seqid", "start", "stop", "strand"], data=inter_genic)
intergen_tab.write(folder_out + "chrom" + str(chrom_coord) + "-intergenic.tsv")
intergen_tab

homo_sapiens,22,0,10524343,1
homo_sapiens,22,10529164,10736169,1
homo_sapiens,22,10736283,10742048,1
homo_sapiens,22,10754413,10939386,1
homo_sapiens,22,10961693,11066266,1
...,...,...,...,...
homo_sapiens,22,50653123,50664424,1
homo_sapiens,22,50733212,50735811,1
homo_sapiens,22,50738173,50738194,1
homo_sapiens,22,50745339,50754673,1
homo_sapiens,22,50755434,50756829,1


In [6]:
pandastable = intergen_tab.to_pandas()
elements_lower = pandastable['start'] > pandastable['stop']
pandastable[elements_lower]

,species,seqid,start,stop,strand


In [7]:
target_start = 36510854 
diff = np.abs(pandastable['start'] - target_start)
closest_index = diff.idxmin()
pandastable.iloc[closest_index]

species    homo_sapiens
seqid                22
start          36507221
stop           36510853
strand                1
Name: 511, dtype: object